# Hyperparameter Tuning using HyperDrive

In [1]:
import joblib

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'sleep-health-project-hyperdrive'

exp = Experiment(ws, experiment_name)

In [3]:
cluster_name = "sleep-health-compute"

# Verfiy that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The Sleep Health and Lifestyle Dataset from Kaggle is used to perform a classification task. The data covers a wide range of variables related to sleep and daily habits. In the classification task it should be determined wether a person has a certain sleep disorder or none.

In [4]:
dataset = Dataset.get_by_name(ws, name='Sleep-Health-Dataset')
df = dataset.to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "learning_rate": choice(0.01, 0.05, 0.1, 0.2, 0.3),
    "max_depth": choice(1, 3, 5, 7, 9),
})

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=3,
    slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
# sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

args = ["--learning_rate", "0.1", "--max_depth", "3"]

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    arguments=args,
    compute_target=cluster)#,
#     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=3)

In [6]:
hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the best run's metrics
best_run_metrics = best_run.get_metrics()

# Get best run's file name
best_run.get_file_names()

In [ ]:
best_run

In [ ]:
# Save best model
filename = "model_1000_10.0.pkl"
best_run.download_file(name=filename, output_file_path=filename)

In [ ]:
model = Model.register(model_path=filename,model_name="best_hyper_model",workspace=ws)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

